# Imports

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as mae, accuracy_score, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from preprocess import preprocess, process
import tensorflow as tf
from tensorflow import keras

# Data

In [5]:
data = pd.read_csv('jobfair_train.csv')

In [7]:
data.head()

,season,club_id,league_id,dynamic_payment_segment,cohort_season,avg_age_top_11_players,avg_stars_top_11_players,avg_stars_top_14_players,avg_training_factor_top_11_players,days_active_last_28_days,...,registration_country,registration_platform_specific,league_match_won_count_last_28_days,training_count_last_28_days,global_competition_level,tokens_spent_last_28_days,tokens_stash,rests_stash,morale_boosters_stash,league_rank
0,173,6042825,2912348,2) Minnow,134,25,5.255151,4.483009,1.778801,25,...,Portugal,Flash FB Canvas,24,435,11.0,598,245,186,818,2
1,173,9620967,2912140,2) Minnow,28,23,4.962521,4.785648,-0.002687,28,...,Turkey,Android Phone,19,58,7.0,269,0,35,24,2
2,173,6045474,2912933,4) Whale,136,23,11.105782,10.511133,0.628794,28,...,Belgium,Flash FB Canvas,26,116,8.0,23180,369,444,4933,1
3,173,14187773,2910371,0) NonPayer,2,21,4.164848,4.023486,0.551904,12,...,Malaysia,Android Phone,11,22,2.0,41,43,213,126,6
4,173,13694853,2907632,0) NonPayer,5,22,3.031939,2.926029,0.690544,0,...,Italy,iOS Phone,9,0,4.0,0,16,100,147,10


# Data split

In [8]:
X = data.drop('league_rank', axis=1)
y = data['league_rank']-1

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
standard_scaler = StandardScaler()

In [11]:
X_train = standard_scaler.fit_transform(preprocess(X_train, y_train))
X_test = standard_scaler.transform(process(X_test))

In [15]:
X_train.shape

(44251, 36)

# Model evaluation

In [27]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(36,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(14, activation='softmax')
])

In [32]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy', 'mean_absolute_error'])

In [33]:
with tf.device('GPU'):
    history = model.fit(X_train, y_train, validation_split=0.1, epochs=100)

Epoch 1/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.4275 - accuracy: 0.1379 - mean_absolute_error: 6.4439 - val_loss: 2.3406 - val_accuracy: 0.1573 - val_mean_absolute_error: 6.4517
Epoch 2/100
1245/1245 [==============================] - 9s 8ms/step - loss: 2.3493 - accuracy: 0.1603 - mean_absolute_error: 6.4439 - val_loss: 2.3280 - val_accuracy: 0.1597 - val_mean_absolute_error: 6.4517
Epoch 3/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.3363 - accuracy: 0.1616 - mean_absolute_error: 6.4439 - val_loss: 2.3313 - val_accuracy: 0.1568 - val_mean_absolute_error: 6.4517
Epoch 4/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.3265 - accuracy: 0.1641 - mean_absolute_error: 6.4439 - val_loss: 2.3197 - val_accuracy: 0.1554 - val_mean_absolute_error: 6.4517
Epoch 5/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.3191 - accuracy: 0.1644 - mean_absolute_error: 6.4439 - val_loss: 2.3102 - val_accur

Epoch 40/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2479 - accuracy: 0.1889 - mean_absolute_error: 6.4439 - val_loss: 2.3179 - val_accuracy: 0.1699 - val_mean_absolute_error: 6.4517
Epoch 41/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2467 - accuracy: 0.1874 - mean_absolute_error: 6.4439 - val_loss: 2.3099 - val_accuracy: 0.1740 - val_mean_absolute_error: 6.4517
Epoch 42/100
1245/1245 [==============================] - 9s 8ms/step - loss: 2.2465 - accuracy: 0.1873 - mean_absolute_error: 6.4439 - val_loss: 2.3100 - val_accuracy: 0.1604 - val_mean_absolute_error: 6.4517
Epoch 43/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2456 - accuracy: 0.1893 - mean_absolute_error: 6.4439 - val_loss: 2.3106 - val_accuracy: 0.1769 - val_mean_absolute_error: 6.4517
Epoch 44/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2448 - accuracy: 0.1895 - mean_absolute_error: 6.4439 - val_loss: 2.3090 - v

Epoch 79/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2210 - accuracy: 0.1982 - mean_absolute_error: 6.4439 - val_loss: 2.3351 - val_accuracy: 0.1645 - val_mean_absolute_error: 6.4517
Epoch 80/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2198 - accuracy: 0.1980 - mean_absolute_error: 6.4439 - val_loss: 2.3387 - val_accuracy: 0.1618 - val_mean_absolute_error: 6.4517
Epoch 81/100
1245/1245 [==============================] - 10s 8ms/step - loss: 2.2208 - accuracy: 0.1968 - mean_absolute_error: 6.4439 - val_loss: 2.3285 - val_accuracy: 0.1674 - val_mean_absolute_error: 6.4517
Epoch 82/100
1245/1245 [==============================] - 9s 8ms/step - loss: 2.2190 - accuracy: 0.1986 - mean_absolute_error: 6.4439 - val_loss: 2.3404 - val_accuracy: 0.1670 - val_mean_absolute_error: 6.4517
Epoch 83/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.2203 - accuracy: 0.1997 - mean_absolute_error: 6.4439 - val_loss: 2.3390 - va

In [34]:
model.compile(optimizer='adam',
             loss='mean_absolute_error',
             metrics=['accuracy', 'mean_absolute_error'])

In [35]:
with tf.device('GPU'):
    history = model.fit(X_train, y_train, validation_split=0.1, epochs=100)

Epoch 1/100
1245/1245 [==============================] - 10s 8ms/step - loss: 6.4439 - accuracy: 0.2032 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1701 - val_mean_absolute_error: 6.4517
Epoch 2/100
1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2033 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1701 - val_mean_absolute_error: 6.4517
Epoch 3/100
1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2033 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1704 - val_mean_absolute_error: 6.4517
Epoch 4/100
1245/1245 [==============================] - 10s 8ms/step - loss: 6.4439 - accuracy: 0.2034 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1704 - val_mean_absolute_error: 6.4517
Epoch 5/100
1245/1245 [==============================] - 8s 7ms/step - loss: 6.4439 - accuracy: 0.2031 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accu

1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2033 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1699 - val_mean_absolute_error: 6.4517
Epoch 41/100
1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2033 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1695 - val_mean_absolute_error: 6.4517
Epoch 42/100
1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2030 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1697 - val_mean_absolute_error: 6.4517
Epoch 43/100
1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2031 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1699 - val_mean_absolute_error: 6.4517
Epoch 44/100
1245/1245 [==============================] - 9s 8ms/step - loss: 6.4439 - accuracy: 0.2029 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.16

Epoch 79/100
1245/1245 [==============================] - 10s 8ms/step - loss: 6.4439 - accuracy: 0.2024 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1710 - val_mean_absolute_error: 6.4517
Epoch 80/100
1245/1245 [==============================] - 9s 7ms/step - loss: 6.4439 - accuracy: 0.2026 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1713 - val_mean_absolute_error: 6.4517
Epoch 81/100
1245/1245 [==============================] - 10s 8ms/step - loss: 6.4439 - accuracy: 0.2024 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1701 - val_mean_absolute_error: 6.4517
Epoch 82/100
1245/1245 [==============================] - 9s 8ms/step - loss: 6.4439 - accuracy: 0.2027 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - val_accuracy: 0.1701 - val_mean_absolute_error: 6.4517
Epoch 83/100
1245/1245 [==============================] - 10s 8ms/step - loss: 6.4439 - accuracy: 0.2027 - mean_absolute_error: 6.4439 - val_loss: 6.4517 - va

In [36]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(36,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1, activation=None)
])

In [37]:
model.compile(optimizer='adam',
             loss='mean_absolute_error',
             metrics=['mean_absolute_error'])

In [38]:
with tf.device('GPU'):
    history = model.fit(X_train, y_train, validation_split=0.1, epochs=100)

Epoch 1/100
1245/1245 [==============================] - 11s 8ms/step - loss: 2.6185 - mean_absolute_error: 2.6185 - val_loss: 2.4077 - val_mean_absolute_error: 2.4077
Epoch 2/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.4391 - mean_absolute_error: 2.4391 - val_loss: 2.3644 - val_mean_absolute_error: 2.3644
Epoch 3/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.4139 - mean_absolute_error: 2.4139 - val_loss: 2.3863 - val_mean_absolute_error: 2.3863
Epoch 4/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.3972 - mean_absolute_error: 2.3972 - val_loss: 2.4164 - val_mean_absolute_error: 2.4164
Epoch 5/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.3803 - mean_absolute_error: 2.3803 - val_loss: 2.3555 - val_mean_absolute_error: 2.3555
Epoch 6/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.3655 - mean_absolute_error: 2.3655 - val_loss: 2.3539 - val_mean_absolute_error: 2.35

Epoch 50/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.1957 - mean_absolute_error: 2.1957 - val_loss: 2.3220 - val_mean_absolute_error: 2.3220
Epoch 51/100
1245/1245 [==============================] - 9s 8ms/step - loss: 2.1934 - mean_absolute_error: 2.1934 - val_loss: 2.3198 - val_mean_absolute_error: 2.3198
Epoch 52/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.1906 - mean_absolute_error: 2.1906 - val_loss: 2.3209 - val_mean_absolute_error: 2.3209
Epoch 53/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.1893 - mean_absolute_error: 2.1893 - val_loss: 2.3321 - val_mean_absolute_error: 2.3321
Epoch 54/100
1245/1245 [==============================] - 9s 8ms/step - loss: 2.1877 - mean_absolute_error: 2.1877 - val_loss: 2.3252 - val_mean_absolute_error: 2.3252
Epoch 55/100
1245/1245 [==============================] - 9s 8ms/step - loss: 2.1839 - mean_absolute_error: 2.1839 - val_loss: 2.3159 - val_mean_absolute_error:

Epoch 99/100
1245/1245 [==============================] - 9s 7ms/step - loss: 2.1287 - mean_absolute_error: 2.1287 - val_loss: 2.3370 - val_mean_absolute_error: 2.3370
Epoch 100/100
1245/1245 [==============================] - 8s 7ms/step - loss: 2.1321 - mean_absolute_error: 2.1321 - val_loss: 2.3347 - val_mean_absolute_error: 2.3347


In [39]:
mae(y_test, model.predict(X_test))

346/346 [==============================] - 1s 3ms/step


2.4208631454298124